[Reference](https://medium.com/python-in-plain-english/seven-intermediate-level-tips-and-tricks-for-python-lists-a81876ef6f33)

# 1. Ensure that a List Contains Unique Elements

In [1]:
from collections import UserList

class UniquesList(UserList):
    """
    A List Class which works just like a list, except
    that it only holds unique values - similar to a set.
    >>> ul = UniquesList("The Jolly Green Giant")
    >>> print("".join(ul))
    The JolyGrniat
    """
    def __init__(self, initlist=None):
        """__init__.
        Args:
            initlist:
        """
        self.data = []

        if initlist:
            if isinstance(initlist, UniquesList):
                self.data[:] = initlist.data[:]
            else:
                for k in initlist:
                    self.append(k)

    def append(self, item) -> None:
        """Append an item to the end of the list.
        Args:
            item: Only unique values are appended, duplicates are omitted
        Returns:
            None:
        """
        if not self.data.count(item):
            super(UniquesList, self).append(item)

dl = UniquesList()
dl.append("Text Value One")
dl.append("Text Value One")
dl.append("Text Value One")
dl.append("Text Value One")
dl.append("Text Value Two")
dl.append("Text Value Two")
dl.append("Text Value Two")
dl.append("Text Value Two")
assert len(dl) == 2

dl = UniquesList()
for i in range(1000):
    dl.append("a")
assert len(dl) == 1

# 2. Find all the Index Values of a Matching a Test Condition

In [2]:
import typing

def get_indices(the_list: list, test_value: object) -> typing.Iterable[int]:
    """
    Returns the indices of matching list items.
    Uses a generator to create an iterator.
    Args:
        the_list: the list containing search elements
        test_value: what we want to find
    Returns: the index of matching list items
    >>> print(list(get_indices("The jolly green giant", "e")))
    [2, 12, 13]
    """

    generator = (key for key, val in enumerate(the_list) if test_value == val)
    for key in generator:
        yield key

# 3. Flatten a List of Lists into one Super List

In [3]:
from itertools import chain


def flatten_nested_lists(*input_list: list) -> list:
    """flatten_nested_lists.
    Args:
        input_list:
    Returns:
        list:
    >>> l1: list = []
    >>> l1.append([1, "2", 3])
    >>> l1.append([4, 5, 6, 7])
    >>> l1.append(["Eight", {"this one": 9}])
    >>> l1.append([10, 11, 12])
    >>> print(list(flatten_nested_lists(*l1)))
    [1, '2', 3, 4, 5, 6, 7, 'Eight', {'this one': 9}, 10, 11, 12]
    """
    for i in chain.from_iterable(input_list):
        yield i
l2: list = []
l2.append([1, 2, 3])
l2.append([4, 5, 6])
l2.append([10, 11, 12])

for list_item in flatten_nested_lists(*l2):
    print(list_item)

1
2
3
4
5
6
10
11
12


# 4. Implement a FrozenList

In [4]:
from collections.abc import Iterable
from collections import UserList


def immutable_decorator(f):
    def wrapper(self, *args, **kwargs):
        raise TypeError("Object is frozen")
    return wrapper

class FrozenList(UserList):  # pylint: disable=too-many-ancestors
    """
    A List which is immutable.
    >>> fl: FrozenList = FrozenList("hello")
    >>> fl:FrozenList = FrozenList([1, 2, 4])
    >>> print(fl[1:2])
    [2]
    >>> print(fl)
    [1, 2, 4]
    >>> fl.append(1)
    Traceback (most recent call last):
     ...
    TypeError: Object is frozen
    >>> fl.extend(1)
    Traceback (most recent call last):
     ...
    TypeError: Object is frozen
    """
    @immutable_decorator
    def __setitem__(self, i: int, o) -> None:
        pass

    @immutable_decorator
    def __add__(self, other):
        pass

    @immutable_decorator
    def __iadd__(self, other):
        pass

    @immutable_decorator
    def __mul__(self, n: int):
        pass

    @immutable_decorator
    def __imul__(self, n: int):
        pass

    @immutable_decorator
    def append(self, item) -> None:
        pass

    @immutable_decorator
    def insert(self, i: int, item) -> None:
        pass

    @immutable_decorator
    def pop(self, i: int):
        pass

    @immutable_decorator
    def remove(self, item) -> None:
        pass

    @immutable_decorator
    def clear(self) -> None:
        pass

    @immutable_decorator
    def reverse(self) -> None:
        pass

    @immutable_decorator
    def extend(self, other) -> None:
        pass

l: list = [1, 2, 4]
fl: FrozenList = FrozenList(l)
assert fl[1:2] == [2]
fl: FrozenList = FrozenList("help")
assert fl[1::2] == ["e", "p"]

# 5. Create an Autoappend List

In [5]:
from collections import UserList

class AutoAppendList(UserList):
    """
    AutoAppendList. Will append an item if you are off by one index assignment.
    >>> aal: AutoAppendList = AutoAppendList()
    >>> for i in range(10):
    ...     aal[i] = i
    >>> print(aal)
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    """
    def __setitem__(self, index, item):
        """__setitem__.
        Args:
            index:
            item:
        """
        if index == len(self.data):
            self.data.append(item)
        else:
            self.data[i] = item

aal: AutoAppendList = AutoAppendList()

for iteration, char in enumerate("hello"):
    aal[iteration] = ord(char)

assert aal == [104, 101, 108, 108, 111]

# 6. Create a List Which Only Accepts Specific Object Types

In [6]:
from collections import UserList
from urllib.parse import urlparse
import json
import requests

def recursive_key_values(dictionary):
    for key, value in dictionary.items():
        i = 0
        if type(value) is str:
            yield (key, value)
        elif type(value) is dict:
            yield from recursive_key_values(value)
        elif type(value) in (list, tuple, set):
            for seq_item in value:
                yield from recursive_key_values({f"{key}_{str(i)}": seq_item})
                i = i + 1
        else:
            yield (key, str(value))

class URLFilteredList(UserList):
    """
    URLFilteredList. Will only accept URLs via append.
    """
    def __init__(self):
        self.data = []

    def append(self, item) -> None:
        if self._is_url(item):
            super().append(item)

    def __setitem__(self, i: int, item):
        if self._is_url(item):
            super().append(item)

    @staticmethod
    def _is_url(value: str) -> bool:
        if value and isinstance(value, str):
            validation = urlparse(value)
            if all([validation.scheme, validation.netloc]):
                return True

        return False

dict1 = dict(
    json.loads(
        requests.get("http://ergast.com/api/f1/2014/5/results.json").text))

ul: URLFilteredList = URLFilteredList()
for k, v in recursive_key_values(dict1):
    ul.append(v)

assert "http://en.wikipedia.org/wiki/2014_Spanish_Grand_Prix" in ul
assert "http://en.wikipedia.org/wiki/Daniel_Ricciardo" in ul
ul[0] = "definitely not a url"
assert ul[0] == 'http://ergast.com/mrd/1.4'

print(ul)

['http://ergast.com/mrd/1.4', 'http://ergast.com/api/f1/2014/5/results.json', 'http://en.wikipedia.org/wiki/2014_Spanish_Grand_Prix', 'http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya', 'http://en.wikipedia.org/wiki/Lewis_Hamilton', 'http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One', 'http://en.wikipedia.org/wiki/Nico_Rosberg', 'http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One', 'http://en.wikipedia.org/wiki/Daniel_Ricciardo', 'http://en.wikipedia.org/wiki/Red_Bull_Racing', 'http://en.wikipedia.org/wiki/Sebastian_Vettel', 'http://en.wikipedia.org/wiki/Red_Bull_Racing', 'http://en.wikipedia.org/wiki/Valtteri_Bottas', 'http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering', 'http://en.wikipedia.org/wiki/Fernando_Alonso', 'http://en.wikipedia.org/wiki/Scuderia_Ferrari', 'http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen', 'http://en.wikipedia.org/wiki/Scuderia_Ferrari', 'http://en.wikipedia.org/wiki/Romain_Grosjean', 'http://en.wikipedia.org/wiki/Lo

# 7. Use Map and Reduce on Lists

In [7]:
import functools

a = [5, 7, 9, 11]
b = [1, 20, 30, 40]
c = [3, 3, 3, 6]

answers = list(map(lambda x, y, z: (x + y) / z, a, b, c))
assert answers == [2.0, 9.0, 13.0, 8.5]

summation = functools.reduce(lambda x, next_reduce: x + next_reduce, [2.0, 9.0, 13.0, 8.5])
assert summation == 32.5